# Data Prep

In [1]:
import pandas as pd
import numpy as np
from PIL import Image
import cv2

In [2]:
all_ann = pd.read_csv("lib/datasets/LISA/allAnnotations.csv", delimiter=';')
all_ann

,Filename,Annotation tag,Upper left corner X,Upper left corner Y,Lower right corner X,Lower right corner Y,"Occluded,On another road",Origin file,Origin frame number,Origin track,Origin track frame number
0,aiua120214-0/frameAnnotations-DataLog02142012_...,stop,862,104,916,158,"0,0",aiua120214-0/DataLog02142012_external_camera.avi,2667,stop_1330545910.avi,2
1,aiua120214-0/frameAnnotations-DataLog02142012_...,speedLimitUrdbl,425,197,438,213,"0,0",aiua120214-0/DataLog02142012_external_camera.avi,2667,stop_1330545910.avi,2
2,aiua120214-0/frameAnnotations-DataLog02142012_...,stop,922,88,982,148,"1,0",aiua120214-0/DataLog02142012_external_camera.avi,2672,stop_1330545910.avi,7
3,aiua120214-0/frameAnnotations-DataLog02142012_...,speedLimit25,447,193,461,210,"0,0",aiua120214-0/DataLog02142012_external_camera.avi,2672,stop_1330545910.avi,7
4,aiua120214-0/frameAnnotations-DataLog02142012_...,speedLimit25,469,189,483,207,"0,0",aiua120214-0/DataLog02142012_external_camera.avi,2677,stop_1330545910.avi,12
...,...,...,...,...,...,...,...,...,...,...,...
7850,vid9/frameAnnotations-MVI_0121.MOV_annotations...,speedLimit35,41,209,65,239,"0,0",vid9/MVI_0121.MOV,8813,speedLimit_1324866802.avi,22
7851,vid9/frameAnnotations-MVI_0121.MOV_annotations...,speedLimit35,526,213,543,233,"0,0",vid9/MVI_0121.MOV,8875,speedLimit_1324866807.avi,2
7852,vid9/frameAnnotations-MVI_0121.MOV_annotations...,speedLimit35,546,208,564,230,"0,0",vid9/MVI_0121.MOV,8880,speedLimit_1324866807.avi,7
7853,vid9/frameAnnotations-MVI_0121.MOV_annotations...,speedLimit35,573,204,592,228,"0,0",vid9/MVI_0121.MOV,8885,speedLimit_1324866807.avi,12


In [3]:
#max image size
height = 50
width = 50
data = []
labels = []
path = "lib/datasets/LISA/"
test = 0
for i in range(len(all_ann["Filename"])):
    file = all_ann["Filename"][i]
    sign = all_ann["Annotation tag"][i]
    image = Image.open(path+file)
    image = image.resize((width,height))
    image = np.asarray(image)
    data.append(image)
    labels.append(sign)


signs = np.array(data)
labels = np.array(labels)

# Randomize order
s = np.arange(signs.shape[0])
np.random.seed(43)
np.random.shuffle(s)
signs = signs[s]
labels = labels[s]

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(signs, labels, test_size=0.2, random_state=1)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=1)

# Preprocess
X_train = np.reshape(X_train, (X_train.shape[0], -1)).astype('float64')
X_val = np.reshape(X_val, (X_val.shape[0], -1)).astype('float64')
X_test = np.reshape(X_test, (X_test.shape[0], -1)).astype('float64')

# Normalize
mean_image = np.mean(X_train, axis = 0)
X_train -= mean_image
X_val -= mean_image
X_test -= mean_image

# Encode
lab = np.unique(labels)

In [4]:
np.unique(labels)

array(['addedLane', 'curveLeft', 'curveRight', 'dip', 'doNotEnter',
       'doNotPass', 'intersection', 'keepRight', 'laneEnds', 'merge',
       'noLeftTurn', 'noRightTurn', 'pedestrianCrossing',
       'rampSpeedAdvisory20', 'rampSpeedAdvisory35',
       'rampSpeedAdvisory40', 'rampSpeedAdvisory45',
       'rampSpeedAdvisory50', 'rampSpeedAdvisoryUrdbl',
       'rightLaneMustTurn', 'roundabout', 'school', 'schoolSpeedLimit25',
       'signalAhead', 'slow', 'speedLimit15', 'speedLimit25',
       'speedLimit30', 'speedLimit35', 'speedLimit40', 'speedLimit45',
       'speedLimit50', 'speedLimit55', 'speedLimit65', 'speedLimitUrdbl',
       'stop', 'stopAhead', 'thruMergeLeft', 'thruMergeRight',
       'thruTrafficMergeLeft', 'truckSpeedLimit55', 'turnLeft',
       'turnRight', 'yield', 'yieldAhead', 'zoneAhead25', 'zoneAhead45'],
      dtype='<U22')